# Setup

In [1]:
import gym
import tensorflow as tf
import retro

import numpy as np
import random as r

from skimage import transform # to preprocess game frames
from skimage.color import rgb2gray # to convert game frames into grayscale

import matplotlib.pyplot as plt

from collections import deque # to create ordered collections of frames

import warnings # ignore warning messages from skiimage during training

warnings.filterwarnings("ignore")

# Create Env

In [2]:
env = gym.make('SpaceInvaders-v0') # make env

print("Frame Size: ", env.observation_space)
print("Action Space: ", env.action_space.n, " moves")

H = 110 # height of preprocessed image
W = 84 # width of preprocessed image

moves = np.array(np.identity(env.action_space.n, dtype=int).tolist())

print(moves)

Frame Size:  Box(210, 160, 3)
Action Space:  6  moves
[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


In [3]:
for i_episode in range(1):
    observation = env.reset()
    print("input size", observation.shape)
#     for t in range(1):
#         env.render()
#         print(observation)
#         action = env.action_space.sample()
#         observation, reward, done, info = env.step(action)
#         if done:
#             print("Episode finished after {} timesteps".format(t+1))
#             break

input size (210, 160, 3)


# Hyperparameters

In [4]:
# model
STATE_SIZE = [110, 84, 4] # stack dimensions
STACK_SIZE = 4 # "phi length" - how many frames we consolidate into one training example
ACTION_SIZE = env.action_space.n # 6 possible moves
LEARNING_RATE = 2.5e-4

# training
NUM_GAMES = 1 # how many games we will play for training
MAX_STEPS = 50000 # max number of moves we'll make each game
HIDDEN_UNITS = 512
BATCH_SIZE = 1 # num of states we include for each weight update in gradient descent
TRAINING = True # when you just want agent to play
ENV_RENDER = False # when you want to see env

# explore params
EXP_START = 1.0
EXP_STOP = 0.01
DECAY = 1e-5

# q le table
GAMMA = 0.001 # discount for future rewards in bellman equation

# memory
LOAD_MEM = 64 # num experiences to be stored in memory during initialization
MEMORY_SIZE = int(1e6) # max num experiences stored in memory

# Preprocess raw input from game

In [5]:
def state_to_frame(state):
    
#     print("Input Size: ", state.shape)
    
    gray_doe = rgb2gray(state)
    crop_doe = gray_doe[6:-12, 4:-12]
    normalize_doe = crop_doe / 255.0
    frame = transform.resize(normalize_doe, [H, W])
    
#     print("Output Size: ", frame.shape)
    
    return  frame # Shape = 108 height, 84 width | vals = [0, 1]

In [6]:
def stack_frames(stack_o_frames, state, is_new_game):
    
    frame = state_to_frame(state)
    
    if is_new_game: # if new game, clear stack from last game
        
        for i in range(STACK_SIZE): # in new game, start with copies of OG frame
            stack_o_frames.append(frame)
            
#         print("Stack Size: ", len(stack_o_frames))  
#         print("State Size: ", state.shape)
        
    else: # if we already have stack, append latest frame
        
        stack_o_frames.append(frame) # deque automatically removes oldest frame
        
    state = np.stack(stack_o_frames, axis=2) # input to neural net, shape = (108, 84, 4)
       
#     print("Snapshot: ", state[:5])
        
    return stack_o_frames, state

# Model

In [7]:
class DQN:
    def __init__(self, name="DQN"):
        
        with tf.variable_scope(name): # create model skeleton, using placeholders for data to be fed in
            
            self.inputs_ = tf.placeholder(dtype=tf.float32, 
                                          shape=[None, *STATE_SIZE], 
                                          # *STATE_SIZE means we take each element of STATE_SIZE and input sequentially
                                          # ie. [None, 110, 84, 4] instead of [None, [110, 84, 4]]
                                          name="inputs")
            
            self.actions_ = tf.placeholder(dtype=tf.float32,
                                           shape=[None, ACTION_SIZE],
                                           name="actions")
            
            # Target QVal = Reward (state | actions) + discount * QVal (new_state | total_possible_actions)
            self.target_q = tf.placeholder(dtype=tf.float32,
                                           shape=[None],
                                           name="target_q")
            
            # First Convolution
            self.conv1 = tf.layers.conv2d(inputs=self.inputs_,
                                          filters=32,
                                          kernel_size=[8,8],
                                          strides=[4,4],
                                          padding="VALID",
                                          kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                          name = "conv1")
            
            # Output size = (W−K+2P)/S+1
            # height = (108 - 8 + 2*0) / 4 + 1= 26
            # width = (84 - 8) / 4 + 1= 20
            
            self.conv1_shape = self.conv1.get_shape()
            
            self.conv1_out = tf.nn.elu(self.conv1, name="conv1_out") # shape = (batch_size, 26 ht, 20 wd, 32 filters)
            
            # Second Convolution
            self.conv2 = tf.layers.conv2d(inputs=self.conv1_out,
                                          filters=64,
                                          kernel_size=[4,4],
                                          strides=[2,2],
                                          padding="VALID",
                                          kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                          name = "conv2")
            
            self.conv2_shape = self.conv2.get_shape()
            
            self.conv2_out = tf.nn.elu(self.conv2, name="conv2_out") # shape = (batch_size, 12 ht, 9 wd, 64 filters)
            
            # Third Convolution
            self.conv3 = tf.layers.conv2d(inputs=self.conv2_out,
                                          filters=64,
                                          kernel_size=[3,3],
                                          strides=[2,2],
                                          padding="VALID",
                                          kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                          name = "conv3")
            
            self.conv3_shape = self.conv2.get_shape()
            
            self.conv3_out = tf.nn.elu(self.conv3, name="conv3_out") # shape = (batch size, 12 ht, 9 wd, 64 filters)
            
            # Flat
            self.flat = tf.layers.flatten(self.conv3_out)
            
            self.flat_shape = self.flat.get_shape()
            
            # Hidden Layer
            self.fully_connected = tf.layers.dense(inputs=self.flat,
                                                   units=HIDDEN_UNITS,
                                                   activation=tf.nn.elu,
                                                   kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                                   name="fc1")
            
            self.fully_connected_shape = self.fully_connected.get_shape()
            
            # Linear
            self.output = tf.layers.dense(inputs=self.fully_connected,
                                          units=ACTION_SIZE,
                                          kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                          activation=None)
            
            self.output_shape = self.output.get_shape()
            
            # q val
            self.q = tf.reduce_sum(tf.multiply(self.output, self.actions_))
        
            # loss = (discounted_q_vals_in_future - q_val_of_move)^2
            self.loss = tf.reduce_mean(tf.square(self.target_q - self.q))
            
            # backprop
            self.optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(self.loss)                    
            
    def print_shapes(self):
        print("c1 ", self.conv1_shape)
        print("c2 ", self.conv2_shape)
        print("c3 ", self.conv3_shape)
        print("flat ", self.fully_connected_shape)
        print("fully_connected ", self.fully_connected_shape)
        print("output ", self.output_shape)
            

In [8]:
tf.reset_default_graph()

DQN = DQN()

DQN.print_shapes()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
c1  (?, 26, 20, 32)
c2  (?, 12, 9, 64)
c3  (?, 12, 9, 64)
flat  (?, 512)
fully_connected  (?, 512)
output  (?, 6)


# Memory module

In [9]:
class Memory():
    def __init__(self):
        self.buffer = deque(maxlen = MEMORY_SIZE)
        
    def add(self, experience):
        self.buffer.append(experience)
        
    def sample(self):
        buffer_size = len(self.buffer)
        index = np.random.choice(np.arange(buffer_size),
                                 size=BATCH_SIZE,
                                 replace=False)
        
        return [self.buffer[i] for i in index]

# Load memory

In [10]:
memory = Memory()

state = env.reset() # reset game
stack = deque([np.zeros((H,W), dtype=np.int) for i in range(STACK_SIZE)], maxlen=STACK_SIZE) # load deque with zeros

# load first state, create first stack
stack, state = stack_frames(stack, state, is_new_game=True)

for i in range(LOAD_MEM):
    
    choice = r.randint(1, len(moves)) - 1 # random index
    action = moves[choice] # chosen index
    next_state, reward, done, _logs_ = env.step(choice) # act
    
    env.render()
    
    stack, next_state = stack_frames(stack, next_state, is_new_game=False) # save next state
    
    if done: # when we lose, create a 0-matrix for next_state
        next_state = np.zeros(state.shape)
    
    memory.add((state, action, reward, next_state, done)) # add experience to memory 
    
    state = next_state
    
    if done: # start new game
        state = env.reset()
        stack, state = stack_frames(stack, state, is_new_game=True) 

print("Experiences Loaded: ", len(memory.buffer))

env.close()

Experiences Loaded:  64


# Tensorboard

In [11]:
writer = tf.summary.FileWriter("./tensorboard/dqn/1") # write

tf.summary.scalar("Loss", DQN.loss) 

write_op = tf.summary.merge_all() # merge

# Training Runs

In [12]:
def move(state, moves, training_step):
    
    adventure = r.uniform(0, 1)

    explore_prob = EXP_STOP - (EXP_START - EXP_STOP) * np.exp(-DECAY * training_step)

    if explore_prob > adventure: # explore randomly
        choice = r.randint(1, len(moves)) - 1
        action = moves[choice]
        
    else: # use dqn
        dq_table = sesh.run(DQN.output, feed_dict={DQN.inputs_:state.reshape((1, *state.shape))}) # run graph with state as input
        choice = np.argmax(dq_table) # pick move with highest predicted q val
        action = moves[choice]
        
    return action, explore_prob

In [17]:
def learn(DQN, memory, sesh):

    batch = memory.sample()

    # retrieve minibatches from memory
    states_mini = np.array([states[0] for states in batch], ndmin=3)
    actions_mini = np.array([actions[1] for actions in batch])
    rewards_mini = np.array([rewards[2] for rewards in batch])
    next_states_mini = np.array([next_states[3] for next_states in batch], ndmin=3)
    dones_mini = np.array([dones[4] for dones in batch])
    
    target_qs = []
    
    # predict q vals for next state
    qvals_next_state = sesh.run(DQN.output, feed_dict={DQN.inputs_:next_states_mini})
    
    for i in range(0, len(batch)): # loop through batches
        
        gameover = dones_mini[i]
        
        if gameover: # target_q = reward
            target_qs.append(rewards_mini[i])
        
        else: # target_q = reward + gamma * max_q(s', a')
            target = rewards_mini[i] + GAMMA * np.max(qvals_next_state[i])
            target_qs.append(target)
            
    targets_mini = np.array([target for target in target_qs]) # create minibatch of target q vals
    
    # calculate loss and backprop on minibatches
    loss, _ = sesh.run([DQN.loss, DQN.optimizer], feed_dict={DQN.inputs_:states_mini,
                                                             DQN.target_q:targets_mini,
                                                             DQN.actions_:actions_mini})
    
    # wrte to tensorboard during each minibatch
    summary = sesh.run(write_op, feed_dict={DQN.inputs_:states_mini,
                                            DQN.target_q:targets_mini,
                                            DQN.actions_:actions_mini})
    writer.add_summary(summary, game)
    writer.flush()
    
    return loss

In [20]:
saver = tf.train.Saver() 

if TRAINING == True:
    
    with tf.Session() as sesh:
        
        tf.global_variables_initializer().run()
        
        training_step = 0
        
        for game in range(NUM_GAMES): # play games
            
            # initialize
            game_step = 0
            game_rewards = []
            state = env.reset()
            
            stack, state = stack_frames(stack, state, is_new_game=True)
            
            while game_step < MAX_STEPS:
                
                game_step += 1
                training_step += 1
                
                action, explore_prob = move(state, moves, training_step) # choose move
                
                next_state, reward, done, _logs_ = env.step(np.argmax(action)) # act
                
                if ENV_RENDER: # show games
                    env.render()
                    
                game_rewards.append(reward) # track reward
                
                if done: # we've lost
                    next_state = np.zeros(state.shape)
                    stack, next_state = stack_frames(stack, next_state, is_new_game=False) 
                    step = MAX_STEPS
                    total_reward = np.sum(game_rewards)
                    memory.add((state, action, reward, next_state, done))
                    
                    print('Game: {}'.format(game), 
                          'Total reward: {}'.format(total_reward),
                          'Explore P: {}'.format(explore_prob),
                          'Training Loss {}'.format(loss))
        
                else: # we're still alive!
                    stack, next_state = stack_frames(stack, next_state, is_new_game=False)
                    memory.add((state, action, reward, next_state, done))
                    state = next_state
                
                loss = learn(DQN, memory, sesh)
                
            if game % 5 == 0: # save model every five games
                save_path = saver.save(sesh, './models/model.cpkt')
                print('Checkpoint')

Game: 0 Total reward: 40.0 Explore P: -0.9761958277305628 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9761859658215951 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.976176104011246 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9761662422995145 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9761563806863998 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9761465191719005 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.976136657756016 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.976126796438745 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9761169352200868 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9761070741000403 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9760972130786045 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9760873521557785 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.976077491331561 Training Loss None
Gam

Game: 0 Total reward: 40.0 Explore P: -0.975032802973397 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9750229526946188 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9750131025143429 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9750032524325682 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9749934024492939 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9749835525645189 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9749737027782422 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.974963853090463 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9749540035011801 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9749441540103927 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9749343046180996 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9749244553242999 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9749146061289928 Training Loss None
G

Game: 0 Total reward: 40.0 Explore P: -0.9739597021643429 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9739498626165191 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9739400231670903 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9739301838160555 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9739203445634137 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9739105054091639 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9739006663533051 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9738908273958364 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9738809885367569 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9738711497760654 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.973861311113761 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9738514725498428 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9738416340843097 Training Loss None


Game: 0 Total reward: 40.0 Explore P: -0.9727600032928166 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9727501757419216 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9727403482893014 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9727305209349555 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9727206936788825 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9727108665210815 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9727010394615518 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.972691212500292 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9726813856373014 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9726715588725791 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9726617322061236 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9726519056379346 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9726420791680106 Training Loss None


Game: 0 Total reward: 40.0 Explore P: -0.9716893784857481 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9716795616410476 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.971669744894515 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9716599282461493 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9716501116959498 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9716402952439152 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9716304788900445 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.971620662634337 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9716108464767915 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9716010304174072 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9715912144561828 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9715813985931177 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9715715828282107 Training Loss None
G

...

Game: 0 Total reward: 40.0 Explore P: -0.9125807842918481 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9125715585301339 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9125623328606771 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9125531072834765 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9125438817985311 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9125346564058401 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9125254311054027 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9125162058972177 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9125069807812844 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9124977557576018 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.912488530826169 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.912479305986985 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9124700812400489 Training Loss None
G

Game: 0 Total reward: 40.0 Explore P: -0.9114927810145619 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9114835661328262 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.911474351343239 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.911465136645799 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9114559220405056 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9114467075273579 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9114374931063549 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9114282787774954 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.911419064540779 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9114098503962044 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9114006363437707 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9113914223834771 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9113822085153228 Training Loss None
Ga

Game: 0 Total reward: 40.0 Explore P: -0.9104152649306015 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9104060608239729 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9103968568093848 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9103876528868364 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9103784490563268 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.910369245317855 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9103600416714202 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9103508381170212 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9103416346546575 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9103324312843278 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9103232280060315 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9103140248197674 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9103048217255348 Training Loss None


Game: 0 Total reward: 40.0 Explore P: -0.9093206222069318 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9093114290466756 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9093022359783505 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9092930430019557 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9092838501174901 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.909274657324953 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9092654646243433 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9092562720156603 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9092470794989027 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9092378870740699 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.909228694741161 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9092195025001747 Training Loss None
Game: 0 Total reward: 40.0 Explore P: -0.9092103103511107 Training Loss None
G

KeyboardInterrupt: 